In [1]:
import pandas as pd
import datetime as datetime
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [2]:
admissions = pd.read_csv("data/ADMISSIONS.CSV")
admissions["ADMITTIME"] = admissions["ADMITTIME"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
admissions["DISCHTIME"] = admissions["DISCHTIME"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
admissions=admissions[["HADM_ID","SUBJECT_ID","ADMITTIME", "DISCHTIME"]]

admissions["window"] = admissions["DISCHTIME"].apply(lambda x: (x,x + datetime.timedelta(days=30)))

In [3]:
def to_set(x):
    return set(x)

sets = admissions.groupby('SUBJECT_ID', as_index = False)['ADMITTIME'].agg({'admitdates': to_set})

In [4]:
admissions = admissions.merge(sets)[["HADM_ID","window","admitdates"]]
admissions.head()

,HADM_ID,window,admitdates
0,165315,"(2196-04-10 15:54:00, 2196-05-10 15:54:00)",{2196-04-09 12:26:00}
1,152223,"(2153-09-08 19:10:00, 2153-10-08 19:10:00)","{2157-10-18 19:34:00, 2153-09-03 07:15:00}"
2,124321,"(2157-10-25 14:00:00, 2157-11-24 14:00:00)","{2157-10-18 19:34:00, 2153-09-03 07:15:00}"
3,161859,"(2139-06-09 12:48:00, 2139-07-09 12:48:00)",{2139-06-06 16:14:00}
4,129635,"(2160-11-05 14:55:00, 2160-12-05 14:55:00)",{2160-11-02 02:06:00}


In [5]:
def fx(x,y):
    for i in y:
        #if i>x[0]+datetime.timedelta(hours=6) and i<x[1]:3393
        if i>x[0] and i<x[1]:
            return 1
    return 0

In [6]:
admissions["test"] = np.vectorize(fx)(admissions["window"],admissions["admitdates"])
admissions.head()

,HADM_ID,window,admitdates,test
0,165315,"(2196-04-10 15:54:00, 2196-05-10 15:54:00)",{2196-04-09 12:26:00},0
1,152223,"(2153-09-08 19:10:00, 2153-10-08 19:10:00)","{2157-10-18 19:34:00, 2153-09-03 07:15:00}",0
2,124321,"(2157-10-25 14:00:00, 2157-11-24 14:00:00)","{2157-10-18 19:34:00, 2153-09-03 07:15:00}",0
3,161859,"(2139-06-09 12:48:00, 2139-07-09 12:48:00)",{2139-06-06 16:14:00},0
4,129635,"(2160-11-05 14:55:00, 2160-12-05 14:55:00)",{2160-11-02 02:06:00},0


In [7]:
sum(admissions["test"])

3469

In [8]:
admissions = admissions[["HADM_ID","test"]]

In [9]:
admissions.to_csv("readmission_labels.csv", index = False)